In [1]:
import openai, os

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":400,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()


In [4]:
query('1+1=')

'2\n\n2+2=4'

In [5]:
overviewForStoryPrompt = """ /* Generate an outline for a story

type character{{
  "name": str,
  "profession": str,
  "age": int,
}}

interface outline {{
  "setting": str, //where the section takes place
  "characters": character[], //the character names
  "conflict": str, //what conflict needs to get solved
  "plotTwist": str, //any plot twists to occur
  "resolution": str // how the story concludes
}}

const generatePlot = (theme, genre, style) : outline =>{{
    return ai.generatePlot(theme)
}}


theme = '{}'
genre = '{}'
style = '{}'
//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(theme, genre, style))
Out:"""

In [6]:
theme = "civil war"
genre = "movie"
style = "shakesepear"


In [7]:
rawOverview = query(overviewForStoryPrompt.format(theme, genre, style), temperature=.5)

In [8]:
rawOverview

'{\n    "setting": "Civil War battlefield",\n    "characters": [\n        {\n            "name": "John",\n            "profession": "soldier",\n            "age": 25\n        },\n        {\n            "name": "William",\n            "profession": "soldier",\n            "age": 27\n        },\n        {\n            "name": "Elizabeth",\n            "profession": "nurse",\n            "age": 30\n        }\n    ],\n    "conflict": "John and William must find a way to survive the battlefield while Elizabeth tries to keep them alive.",\n    "plotTwist": "John and William are brothers who are on opposing sides of the war.",\n    "resolution": "John and William reconcile and Elizabeth helps them escape the battlefield, allowing them to reunite with their family."\n}'

In [9]:
import json

In [10]:
parsedOverview = json.loads(rawOverview)

In [11]:
parsedOverview

{'setting': 'Civil War battlefield',
 'characters': [{'name': 'John', 'profession': 'soldier', 'age': 25},
  {'name': 'William', 'profession': 'soldier', 'age': 27},
  {'name': 'Elizabeth', 'profession': 'nurse', 'age': 30}],
 'conflict': 'John and William must find a way to survive the battlefield while Elizabeth tries to keep them alive.',
 'plotTwist': 'John and William are brothers who are on opposing sides of the war.',
 'resolution': 'John and William reconcile and Elizabeth helps them escape the battlefield, allowing them to reunite with their family.'}

In [12]:
outlineForStoryPrompt = """ /* Generate an outline for a story

type section{{
  "name": str, // the name of the section
  "description": str, // a brief overview of what happens in the section
}}

interface outline {{
  "parts": section[] //all of the sections in the story
}}

const generateOutline = (storyOverview) : outline =>{{
    return ai.generateOutline(storyOverview)
}}

overview = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generateOutline(overview))
Out:"""

In [13]:
rawOutline = query(outlineForStoryPrompt.format(rawOverview), temperature=.5)

In [14]:
rawOutline

'{\n    "parts": [\n        {\n            "name": "Introduction",\n            "description": "Introduce the setting, characters, and conflict."\n        },\n        {\n            "name": "Rising Action",\n            "description": "John and William must find a way to survive the battlefield while Elizabeth tries to keep them alive."\n        },\n        {\n            "name": "Plot Twist",\n            "description": "John and William are revealed to be brothers on opposing sides of the war."\n        },\n        {\n            "name": "Resolution",\n            "description": "John and William reconcile and Elizabeth helps them escape the battlefield, allowing them to reunite with their family."\n        }\n    ]\n}'

In [15]:
parsedOutline = json.loads(rawOutline)

In [16]:
parsedOutline

{'parts': [{'name': 'Introduction',
   'description': 'Introduce the setting, characters, and conflict.'},
  {'name': 'Rising Action',
   'description': 'John and William must find a way to survive the battlefield while Elizabeth tries to keep them alive.'},
  {'name': 'Plot Twist',
   'description': 'John and William are revealed to be brothers on opposing sides of the war.'},
  {'name': 'Resolution',
   'description': 'John and William reconcile and Elizabeth helps them escape the battlefield, allowing them to reunite with their family.'}]}

config file is an object, inputs and outputs

for each input, that's on the bottom, each output on the top

# WE STARTED TO WRITE THIS THEN REALIZED WE SKIPPING AHEAD

In [111]:
promptGenerator = """Generate typescript files to perform a function, e.g.

Purpose: generate a poem
{{
  "description": "generate a poem",
  "shortName": "generatePoem",
  "inputs": [
  {{"style": "shakespearean",
    "interpretationOfStyle": "the author type to emultae"
  }}
  ],
  "outputs": [
    {{
      "poem": "O Romeo, Romeo, wherefore art thou Romeo?\nDeny thy father and refuse thy name."
    }}
  ]
}}

Purpose: {}
"""

In [112]:
query(promptGenerator.format('generate a paragraph'))

'{\n  "description": "generate a paragraph",\n  "shortName": "generateParagraph",\n  "inputs": [\n    {\n      "style": "informative",\n      "interpretationOfStyle": "the type of information to include"\n    }\n  ],\n  "outputs": [\n    {\n      "paragraph": "The human brain is an incredibly complex organ, capable of processing vast amounts of information and making decisions in a fraction of a second. It is composed of billions of neurons, which communicate with each other through electrical and chemical signals. The brain is responsible for controlling all of the body\'s functions, from breathing and digestion to movement and emotion."\n    }\n  ]\n}'

In [17]:
fillOutlinePrompt = """ /* Write the next paragraph in a story for a given purpose

fullOverview = '{}'
fullOutline = '{}'
priorStory = '{}'

const generateNextParagraph = (instructions, currentOutlineSection) : str =>{{
    return ai.write()
}}

instructions = '{}'
currentOutlineSection = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generateNextParagraph(instructions, currentOutlineSection))
Out:"""

In [18]:
story = ""
currentRawOverview = rawOverview
currentRawOutline = rawOutline

In [19]:
parsedOutline['parts'][0]['name']

'Introduction'

In [20]:
curPrompt = fillOutlinePrompt.format(rawOverview, rawOutline, story, "build the setting", parsedOutline['parts'][0]['name'])
print(curPrompt)


 /* Write the next paragraph in a story for a given purpose

fullOverview = '{
    "setting": "Civil War battlefield",
    "characters": [
        {
            "name": "John",
            "profession": "soldier",
            "age": 25
        },
        {
            "name": "William",
            "profession": "soldier",
            "age": 27
        },
        {
            "name": "Elizabeth",
            "profession": "nurse",
            "age": 30
        }
    ],
    "conflict": "John and William must find a way to survive the battlefield while Elizabeth tries to keep them alive.",
    "plotTwist": "John and William are brothers who are on opposing sides of the war.",
    "resolution": "John and William reconcile and Elizabeth helps them escape the battlefield, allowing them to reunite with their family."
}'
fullOutline = '{
    "parts": [
        {
            "name": "Introduction",
            "description": "Introduce the setting, characters, and conflict."
        },
      

In [21]:
curPrompt = fillOutlinePrompt.format(rawOverview, rawOutline, story, "intro", parsedOutline['parts'][0]['name'])

query(curPrompt, temperature=.7)

"John and William were two soldiers on a Civil War battlefield. John was 25 and William was 27. Elizabeth, a nurse, was 30. Their mission was to find a way to survive the battlefield while Elizabeth tried to keep them alive. Little did they know that John and William were brothers on opposing sides of the war. With Elizabeth's help, John and William were able to reconcile and escape the battlefield, allowing them to reunite with their family."

In [22]:
def createPrompt(overview, outline, story, fnName, currentSection):
    fillOutlinePrompt = """ /* Write the next paragraph in a story for a given purpose

    //the overview of the entire story
    fullOverview = {}
    
    //the outline that the story should follow
    fullOutline = {}
    
    //the paragraphs of the story so far
    storySoFar = {}

    const {} = (instructions, currentOutlineSection) : str =>{{
        return ai.write()
    }}

    currentOutlineSection = {}

    console.log("Out: " + {}(currentOutlineSection))
    Out:""".format(overview, outline, story, fnName, currentSection, fnName)
    
    return fillOutlinePrompt

In [23]:
print(createPrompt(rawOverview, rawOutline, story, 'generateIntroduction', parsedOutline['parts'][0]['name']))



 /* Write the next paragraph in a story for a given purpose

    //the overview of the entire story
    fullOverview = {
    "setting": "Civil War battlefield",
    "characters": [
        {
            "name": "John",
            "profession": "soldier",
            "age": 25
        },
        {
            "name": "William",
            "profession": "soldier",
            "age": 27
        },
        {
            "name": "Elizabeth",
            "profession": "nurse",
            "age": 30
        }
    ],
    "conflict": "John and William must find a way to survive the battlefield while Elizabeth tries to keep them alive.",
    "plotTwist": "John and William are brothers who are on opposing sides of the war.",
    "resolution": "John and William reconcile and Elizabeth helps them escape the battlefield, allowing them to reunite with their family."
}
    
    //the outline that the story should follow
    fullOutline = {
    "parts": [
        {
            "name": "Introduction",

In [24]:
r = query(createPrompt(rawOverview, rawOutline, story, 'generateIntroduction', parsedOutline['parts'][0]['name']))

In [25]:
print(r)

It was a chaotic scene on the Civil War battlefield. John, a 25 year old soldier, and William, a 27 year old soldier, were desperately trying to survive the onslaught of bullets and explosions. Elizabeth, a 30 year old nurse, was doing her best to keep them alive. The conflict was clear: John and William had to find a way to survive the battlefield while Elizabeth tried to keep them alive.


In [26]:
story = [r]

In [28]:
r = query(createPrompt(rawOverview, rawOutline, story, 'generateNextParagraph', parsedOutline['parts'][0]['name']))
story.append(r)

In [31]:
print('\n'.join(story))

It was a chaotic scene on the Civil War battlefield. John, a 25 year old soldier, and William, a 27 year old soldier, were desperately trying to survive the onslaught of bullets and explosions. Elizabeth, a 30 year old nurse, was doing her best to keep them alive. The conflict was clear: John and William had to find a way to survive the battlefield while Elizabeth tried to keep them alive.
Little did they know, John and William were brothers on opposing sides of the war. Elizabeth had been trying to keep them both alive, unaware of their familial connection. As the battle raged on, the trio had to find a way to survive and reconcile their differences.


In [48]:
def generateNextStep (rawOverview, rawOutline, story):
    prompt = """//storyRouter.js Generate the name of the function and the inputs and outputs that it will take


story.rawOverview = {}

story.rawOutline = {}

story.current = {}



interface move{{
    "currentStatus": str, //the current status of the story, e.g. "building setting", "building characters", "writing story"
    "storyDirection": str, //the direction the story is going, e.g."building setting", "building down", "building to a climax"
    //the AI is capable of performing any task
    "nextFunction": str, //the name of the function to call, e.g. if building setting it would be "buildSetting", if writing story it would be "writeStory"
    "functionInputs": str, //the inputs the next function will need, e.g. {{character: "Romeo", emotion: "sad"}}; be creative
}}

const generateNextMove = () : move =>{{
    return ai.generateNextMove()
}}
    
console.log('Out: ' + generateNextMove())
//return 
Out:""".format(rawOverview, rawOutline, story)
    return prompt
    

In [49]:
query(generateNextStep(rawOverview, rawOutline, story), temperature=.5)

'{\n    "currentStatus": "building setting",\n    "storyDirection": "building up",\n    "nextFunction": "buildSetting",\n    "functionInputs": {\n        "setting": "Civil War battlefield",\n        "characters": [\n            {\n                "name": "John",\n                "profession": "soldier",\n                "age": 25\n            },\n            {\n                "name": "William",\n                "profession": "soldier",\n                "age": 27\n            },\n            {\n                "name": "Elizabeth",\n                "profession": "nurse",\n                "age": 30\n            }\n        ]\n    }\n}'

In [50]:
story

['It was a chaotic scene on the Civil War battlefield. John, a 25 year old soldier, and William, a 27 year old soldier, were desperately trying to survive the onslaught of bullets and explosions. Elizabeth, a 30 year old nurse, was doing her best to keep them alive. The conflict was clear: John and William had to find a way to survive the battlefield while Elizabeth tried to keep them alive.',
 'Little did they know, John and William were brothers on opposing sides of the war. Elizabeth had been trying to keep them both alive, unaware of their familial connection. As the battle raged on, the trio had to find a way to survive and reconcile their differences.']

In [ ]:
def createPromptV2(overview, outline, story, fnName, currentSection):
    fillOutlinePrompt = """ /* Write the next paragraph in a story for a given purpose

    //the overview of the entire story
    fullOverview = {}
    
    //the outline that the story should follow
    fullOutline = {}
    
    //the paragraphs of the story so far
    storySoFar = {}

    const {} = (props) : str =>{{
        return ai.write()
    }}

    currentOutlineSection = {}

    console.log("Out: " + {}(currentOutlineSection))
    Out:""".format(overview, outline, story, fnName, currentSection, fnName)
    
    return fillOutlinePrompt